In [ ]:
#Scraping Sale section

from datetime import datetime
import json
import requests
import io
response = requests.post('https://app.sakneen.com/apis/marketplace/filters/v2?limit=30&page=1')
x = json.loads(response.text)
pages = x['numberOfPages']
rows=[]
for i in range(1, pages+1):
    response = requests.post('https://app.sakneen.com/apis/marketplace/filters/v2?limit=30&page=' + str(i))
    x = json.loads(response.text)
    rows+=x['data']

dt = datetime.now()
dt = dt.strftime('%Y-%m-%d@%H.%M.%S')
with io.open('Sakneen_Sale_' + str(dt) + '.json', encoding='utf-8', mode = "w") as f:
    f.write("[\n")
    i = 1
    for row in rows:
        x = json.dumps(row, ensure_ascii=False)
        if i != len(rows):
            f.write("\t" + x + ",\n")
        else:
            f.write("\t" + x + "\n")
        i+=1
    f.write("]")
print(len(rows), 'scraped lisitngs.')